In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from sentence_transformers import SentenceTransformer

# 1. Connect to Qdrant
client = QdrantClient("http://localhost:6333")

# 2. Load your embedding model
model = SentenceTransformer("MCINext/Hakim")

# 3. Create a collection
client.recreate_collection(
    collection_name="fa_keywords",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE)
)

# 4. Prepare data
texts = [
    "آیفون ۱۶",
    "آیفون ۱۵",
    "هندزفری",
    "گوشی همراه",
    "سامسونگ a54",
    "طلا",
    "نقره"
]
vectors = model.encode(texts).tolist()

# 5. Upload to Qdrant
client.upsert(
    collection_name="fa_keywords",
    points=[
        {"id": i, "vector": vec, "payload": {"text": text}}
        for i, (text, vec) in enumerate(zip(texts, vectors))
    ]
)

# 6. Query
query = "آیفون"
query_vec = model.encode([query])[0].tolist()

results = client.search(
    collection_name="fa_keywords",
    query_vector=query_vec,
    limit=5
)

for r in results:
    print(r.payload["text"], "→", r.score)
